In [1]:
import pandas as pd
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MIT\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df = pd.read_csv("../../raw_analyst_ratings.csv")
df.dtypes

Unnamed: 0     int64
headline      object
url           object
publisher     object
date          object
stock         object
dtype: object

In [3]:
df.sample()

,Unnamed: 0,headline,url,publisher,date,stock
1050356,1055560,"Marijuana Arrests Declining, But Still High: W...",https://www.benzinga.com/news/16/09/8512190/ma...,Javier Hasse,2016-09-30 00:00:00,QQQ


In [4]:
df.isnull().sum()

Unnamed: 0    0
headline      0
url           0
publisher     0
date          0
stock         0
dtype: int64

**Descriptive Statistics**

In [5]:
def count_words(column):
    return len(column.split())

df['word_count'] = df['headline'].apply(count_words)

In [6]:
df['word_count'].describe().round(2)

count    1407328.00
mean          11.42
std            6.35
min            1.00
25%            7.00
50%           10.00
75%           13.00
max           77.00
Name: word_count, dtype: float64

The average number of words in the headlines for the given dataset is around 11 words. From the summary above, it is also noticeable that there is a headline consisting of a single word to the minimum and there is a headline containing up to 77 words to the maximum.

In [7]:
articles_per_pub = df.groupby('publisher')['headline'].count().reset_index()

In [10]:
sorted_articlesPerPub = articles_per_pub.sort_values(by='headline', ascending=False)
sorted_articlesPerPub.head(10)

,publisher,headline
686,Paul Quintaro,228373
532,Lisa Levin,186979
96,Benzinga Newsdesk,150484
169,Charles Gross,96732
622,Monica Gerson,82380
275,Eddie Staley,57254
353,Hal Lindon,49047
270,ETF Professor,28489
478,Juan Lopez,28438
98,Benzinga Staff,28114


These are the top 10 most active publishers based on the number of articles they publish.

In [20]:
articles_per_date = df.groupby('date')['headline'].count().reset_index()

In [21]:
articles_per_date.sample(5)

,date,headline
12497,2016-10-28 07:01:52-04:00,1
18478,2018-07-26 00:00:00,1125
6345,2015-04-27 08:05:06-04:00,1
2149,2012-08-10 06:46:37-04:00,2
12727,2016-11-11 09:36:53-04:00,1


In [74]:
date_indexed_df = articles_per_date.copy()
date_indexed_df["date"] = pd.to_datetime(date_indexed_df["date"],format='ISO8601',utc=True)

date_indexed_df.set_index("date",inplace=True)
monthly_data = date_indexed_df.resample('ME').sum()
sorted_articlesPerDate = monthly_data.sort_values(by='headline', ascending=False)
sorted_articlesPerDate.head(10)

,headline
date,
2020-03-31 00:00:00+00:00,24995
2020-04-30 00:00:00+00:00,20757
2020-02-29 00:00:00+00:00,18878
2020-05-31 00:00:00+00:00,17992
2018-08-31 00:00:00+00:00,16681
2019-10-31 00:00:00+00:00,16560
2019-08-31 00:00:00+00:00,16462
2018-10-31 00:00:00+00:00,16294
2020-01-31 00:00:00+00:00,16145


The top 2 periods in which news related to the stock market was frequent are March and April of 2020. This may be due to the crash that followed instability due to the COVID-19 pandemic which spanned the period from February 20 to April 7.

**Text Analysis**